In [8]:
import pandas as pd
import numpy as np
import tqdm
import json
import networkx as nx
import collections

### load data

In [6]:
cleaned_recipes = json.load(open("./../generated/high_score_repr_recipes.json"))
repr_per_id = json.load(open("./../generated/high_score_key_representative.json"))

In [7]:
cleaned_recipes[0]

['usda_id=11251',
 'black olife',
 'grape tomato',
 'usda_id=11215',
 'usda_id=11981',
 'purple onion',
 'usda_id=2075',
 'usda_id=16056',
 'usda_id=1019']

#### create graph

In [20]:
def name_ingredients(recipe) :
    print("n_i : ", recipe)
    named_ingredients = []
    
    for r in recipe :
        if r[:9] == "usda_id=" :
            named_ingredients.append(repr_per_id[r[9:]])
            
        else :
            named_ingredients.append(r)
            
    print(named_ingredients)
    return named_ingredients
            

def ingredients_tuples(recipe) :
    print("i_t : ", recipe)
    tuples = []
    for ing1 in range(len(recipe) - 1) :
        for ing2 in range(ing1 + 1, len(recipe)) :
            if recipe[ing1] < recipe[ing2] :
                tuples.append((recipe[ing1], recipe[ing2]))
                
    return tuples
                
asso_counter = collections.Counter()
for recipe in tqdm.tqdm(cleaned_recipes[:1]) :
        asso_counter.update(ingredients_tuples(name_ingredients(recipe)))
            

100%|██████████| 1/1 [00:00<00:00, 994.15it/s]

n_i :  ['usda_id=11251', 'black olife', 'grape tomato', 'usda_id=11215', 'usda_id=11981', 'purple onion', 'usda_id=2075', 'usda_id=16056', 'usda_id=1019']
['usda_id=11251', 'black olife', 'grape tomato', 'usda_id=11215', 'usda_id=11981', 'purple onion', 'usda_id=2075', 'usda_id=16056', 'usda_id=1019']
i_t :  ['usda_id=11251', 'black olife', 'grape tomato', 'usda_id=11215', 'usda_id=11981', 'purple onion', 'usda_id=2075', 'usda_id=16056', 'usda_id=1019']


In [18]:
repr_per_id[str(11251)]

'romaine'

In [16]:
asso_counter.most_common(2)

[(('usda_id=11251', 'usda_id=11981'), 1),
 (('usda_id=11251', 'usda_id=2075'), 1)]